In [19]:
__depends__=[]
__dest__= ["../results/ebtel_varying_tau_results.pickle",
           "../results/ebtel_varying_flux_limiter_results.pickle"
          ]

# Compute EBTEL Results
Run the single- and two-fluid EBTEL models for a variety of inputs. This will be the basis for the rest of our analysis.

First, import any needed modules.

In [20]:
import sys
import os
import pickle

import numpy as np

sys.path.append(os.path.join(os.environ['EXP_DIR'],'EBTEL_analysis/src'))
import ebtel_run as ebr
import ebtel_configure as ebc

Setup the base dictionary for all of the runs.

In [21]:
config_dict = {'usage_option':'no_dem','rad_option':'rk','dem_option':'new','heat_flux_option':'classical',
              'solver':'rka4','ic_mode':'st_eq','print_plasma_params':'True'}
config_dict['total_time'] = 5000
config_dict['tau'] = 0.1
config_dict['rka_error'] = 1.0e-6
config_dict['index_dem'] = 451
config_dict['sat_limit'] = 1.0/6.0
config_dict['h_back'] = 3.5e-5
config_dict['heating_shape'] = 'triangle'
config_dict['t_start_switch'] = 'uniform'
config_dict['t_end_switch'] = 'uniform'
config_dict['T0'] = 1.0e+6
config_dict['n0'] = 1.0e+8
config_dict['t_start'] = 0.0
config_dict['mean_t_start'] = 1000
config_dict['std_t_start'] = 1000
config_dict['amp_switch'] = 'uniform'
config_dict['amp0'] = 0.001
config_dict['amp1'] = 0.01
config_dict['alpha'] = 1.5
config_dict['loop_length'] = 40.0
config_dict['num_events'] = 1
config_dict['t_pulse_half'] = 100
config_dict['h_nano'] = 0.1
config_dict['sample_rate'] = 1
#variable c1 options
config_dict['r3_cond_0'] = 6.0
config_dict['r3_rad_0'] = 0.6
config_dict['r3_loss_correction'] = 'True'
config_dict['r3_grav_correction'] = 'True'
#dummy output file configuration
config_dict['output_file'] = '../results/_tmp_'

Next, construct a function that will make it easy to run all of the different EBTEL configurations.

In [22]:
def run_and_print(tau,h0,L,f,flux_opt,run_inst_sf,run_inst_tf,conf_inst):
    #set params
    conf_inst.config_dictionary['t_pulse_half'] = tau/2.0
    conf_inst.config_dictionary['h_nano'] = h0
    conf_inst.config_dictionary['loop_length'] = L
    conf_inst.config_dictionary['sat_limit'] = f
    conf_inst.config_dictionary['heat_flux_option'] = flux_opt
    #print config file
    conf_inst.print_xml_config(config_file=conf_inst.config_dictionary['output_file']+'.xml')
    #single-fluid    
    #run model
    run_inst_sf.run_ebtel_single(conf_inst.config_dictionary['output_file']+'.xml',verbose=False)
    #save parameters to list
    temp = np.loadtxt(conf_inst.config_dictionary['output_file']+'.txt')
    t,T,n,na = temp[:,0],temp[:,1],temp[:,2],temp[:,6]
    #two-fluid
    #--electron heating
    conf_inst.config_dictionary['heat_species'] = 'electron'
    conf_inst.print_xml_config(config_file=conf_inst.config_dictionary['output_file']+'.xml')
    #run model
    run_inst_tf.run_ebtel_single(conf_inst.config_dictionary['output_file']+'.xml',verbose=False)
    #save parameters to list
    temp = np.loadtxt(conf_inst.config_dictionary['output_file']+'.txt')
    te,Tee,Tei,ne,nea = temp[:,0],temp[:,1],temp[:,2],temp[:,3],temp[:,9]
    #--ion heating
    #print config file
    conf_inst.config_dictionary['heat_species'] = 'ion'
    conf_inst.print_xml_config(config_file=conf_inst.config_dictionary['output_file']+'.xml')
    #run model
    run_inst_tf.run_ebtel_single(conf_inst.config_dictionary['output_file']+'.xml',verbose=False)
    #save parameters to list
    temp = np.loadtxt(conf_inst.config_dictionary['output_file']+'.txt')
    ti,Tie,Tii,ni,nia = temp[:,0],temp[:,1],temp[:,2],temp[:,3],temp[:,9]
    #return dictionary
    return {'t':t,'te':te,'ti':ti,'T':T,'Tee':Tee,'Tei':Tei,'Tie':Tie,'Tii':Tii,'n':n,'ne':ne,'ni':ni,
           'na':na,'nea':nea,'nia':nia,'loop_length':L*1.0e+8,'heat_flux_option':flux_opt}

Configure instances of the single- and two-fluid runners and the instance to print the configuration files.

In [23]:
runner_sf = ebr.Runner(os.path.join(os.environ['EXP_DIR'],'EBTEL_C/bin/'),'')
runner_tf = ebr.Runner(os.path.join(os.environ['EXP_DIR'],'EBTEL-2fluid_repo/bin/'),'')
configurer = ebc.Configurer(config_dict,'')

Finally, run the model for varying pulse duration.

In [24]:
tau_h = [20,40,200,500]
tau_h_results = []
for t in tau_h:
    results = run_and_print(t,20.0/t,40.0,1.0,'limited',runner_sf,runner_tf,configurer)
    tau_h_results.append(results)

And then run the models for varying flux-limiter, $f$.

In [25]:
flux_lim = [{'f':1.0,'opt':'limited','flab':r'$f=1$'},{'f':0.53,'opt':'limited','flab':r'$f=0.53$'},
            {'f':1.0/6.0,'opt':'limited','flab':r'$f=1/6$'},{'f':0.1,'opt':'limited','flab':r'$f=0.1$'},
            {'f':1.0/30.0,'opt':'limited','flab':r'$f=1/30$'},{'f':1.0,'opt':'classical','flab':r'$\mathrm{Spitzer}$'}]
flux_lim_results = []
for i in range(len(flux_lim)):
    results = run_and_print(200.0,0.1,40.0,flux_lim[i]['f'],flux_lim[i]['opt'],runner_sf,runner_tf,configurer)
    flux_lim_results.append(results)

Save both data structures to serialized files.

In [26]:
with open(__dest__[0],'wb') as f:
    pickle.dump(tau_h_results,f)
with open(__dest__[1],'wb') as f:
    pickle.dump(flux_lim_results,f)